 # DA Final project:

# Group 52

# India Agriculture Crop Production

###  Name of students:
Koliya Aharon<br>
Ateret Bass

![indiaagriculture](./india.PNG)

In [ ]:
import pandas as pd
import numpy as np
import plotly.express as px
import seaborn as sns
from IPython.display import Image
from pathlib import Path

In [ ]:
import json
with open("Indian_States.json") as f:
    states = json.load(f)

# Data Preparation

In [ ]:
df = pd.read_csv("India Agriculture Crop Production.csv")

In [ ]:
df.shape

In [ ]:
df.columns

In [ ]:
df.dtypes

In [ ]:
df.isnull().sum().sort_values()

In [ ]:
df.info()

In [ ]:
df = df.dropna()

In [ ]:
def change_year(year):
    first_half = int(year.split("-")[0])
    return str(first_half) + "-" + str(first_half + 1)

In [ ]:
df.Year = df.Year.apply(change_year)

In [ ]:
df = df.sort_values("Year").reset_index(drop = True)

In [ ]:
df.head()

In [ ]:
df.nunique()

In [ ]:
df.describe()

# Crops efficiency

To define efficiency we need to look at the `Yield` column.

Since `Yield` is definde as: 
$$
\text{Yield} = \frac{\text{Production}}{\text{Area}}
$$
Meaning that we calculate how much Crops one unit of `Area` (Hectare) outputs.

For comparision we will look only at crops that the `Production Unit` is `Tonnes`.

In [ ]:
Crop_efficiency_data = df[df["Production Units"] == "Tonnes"]

## Basic Crop efficiency

<div dir="rtl">
מהו היבול הכי יעיל ?
</div>

In [ ]:
crop_efficiency = Crop_efficiency_data.groupby("Crop").Yield.mean().sort_values(ascending = True).to_frame().reset_index()

In [ ]:
fig = px.bar(crop_efficiency, y= "Crop", x= "Yield", title= "Crop average efficiency")
fig.update_layout(
    autosize=False,
    width=800,
    height=1000,
)
fig.show()

<div dir="rtl">
היבול הכי יעיל בממוצע הוא קנה סוכר
</div>

## Crop quantity

<div dir="rtl">
מהו היבול עם התפוקה הגבוהה ביותר ?
</div>

In [ ]:
crop_quantity = Crop_efficiency_data.groupby("Crop").Production.mean().sort_values(ascending = True).to_frame().reset_index()

In [ ]:
fig = px.bar(crop_quantity, y= "Crop", x= "Production", title= "Crop average quantity")
fig.update_layout(
    autosize=False,
    width=800,
    height=1000,
)
fig.show()

<div dir="rtl">
היבול עם הכי הרבה תפוקה בממוצע הוא גם קנה סוכר
</div>

![indiaagriculture](./sugar.PNG)

# Crop Area

<div dir="rtl">
שאלה: האם ליבול היעיל ביותר בממוצע עם התפוקה הגבוהה ביותר בממוצע מוקצה הכי הרבה שטח בממוצע ?
</div>

In [ ]:
crop_area = Crop_efficiency_data.groupby("Crop").Area.mean().sort_values(ascending = True).to_frame().reset_index()

In [ ]:
fig = px.bar(crop_area, y= "Crop", x= "Area", title= "Crop average area")
fig.update_layout(
    autosize=False,
    width=800,
    height=1000,
)
fig.show()

<div dir="rtl">
מסקנה: למרות שקנה סוכר הוא היבול היעיל ביותר בממוצע עם התפוקה הגדולה ביותר בממוצע לא מוקצה לו הכי הרבה שטח בממוצע.
    במקום זה ניתן לראות שדווקא הוקצה הכי הרבה שטח לזרעים לשמן, חיטה ואורז.
    שהם גידוליים בסיסיים לחיי אדם ולכן להם הוקצה את רוב השטח. 
    בנוסף אנחנו רואים שקנה סוכר לא נמצא אפילו בעשירייה הראשונה ועדיין הוא מייצר הכי הרבה תפוקה.
    מה שמחזק את רמת היעילות שלו.
</div>

## Crop efficiency by State

In [ ]:
state_most_efficient_crop = Crop_efficiency_data.groupby(["State"]).apply(lambda x: x.groupby("Crop").Yield.mean().idxmax())
state_most_efficient_crop = state_most_efficient_crop.to_frame().reset_index().rename({0:"Crop"}, axis= 1)

In [ ]:
px.choropleth_mapbox(state_most_efficient_crop, geojson= states,featureidkey= "properties.NAME_1", locations="State",
                     color= "Crop",
                        color_continuous_scale="Viridis",
                           range_color=(0, 12),
                           mapbox_style="carto-positron",
                           zoom=3, center = {"lat": 22.736424, "lon":78.473622},
                           opacity=0.5)

<div dir="rtl">
העשרה: ניתן לראות שקני סוכר הוא גם היבול היעיל ביותר ברוב המדינות
</div>

In [ ]:
del crop_efficiency
del crop_quantity
del crop_area

# States

## State efficiency

<div dir="rtl">
שאלה: מה המדינה שמקצה הכי הרבה שטח ליבול בממוצע ?
</div>

In [ ]:
state_area = Crop_efficiency_data.groupby("State").Area.mean().sort_values(ascending = True).to_frame().reset_index()

In [ ]:
fig = px.bar(state_area, y= "State", x= "Area", title= "Crop average area")
fig.update_layout(
    autosize=False,
    width=800,
    height=1000,
)
fig.show()

In [ ]:
state_area.sort_values("Area", ascending= False).head()

<div dir="rtl">
הטבלה מציגה את חמש המדינות שמקצות הכי הרבה שטח ליבול בממוצע
</div>

In [ ]:
del state_area

<div dir="rtl">
השאלה : האם המדינה שמקצה בממוצע הכי הרבה שטח ליבול היא המדינה הכי יעילה בממוצע ?
</div>

In [ ]:
mean_crop_efficiency_by_state =  Crop_efficiency_data.groupby(["State"]).Yield.mean().to_frame().reset_index()

In [ ]:
px.choropleth_mapbox(mean_crop_efficiency_by_state, geojson= states,featureidkey= "properties.NAME_1", locations="State",
                     color= "Yield",
                        color_continuous_scale="Viridis",
                           range_color=(0, 12),
                           mapbox_style="carto-positron",
                           zoom=3, center = {"lat": 22.736424, "lon":78.473622},
                           opacity=0.5)

In [ ]:
mean_crop_efficiency_by_state.sort_values("Yield", ascending=False).head()

<div dir="rtl">
הטבלה מציגה את חמש המדינות היעלות ביותר בממוצע
</div>

<div dir="rtl">
מסקנה: חמש המדינות היעילות ביותר בממוצע ליבול הן לא המדינות המקצות את ממוצע השטח הגדול ביותר וניתן להבין שאין קשר בין גודל השטח ליעילות שלו 
</div> 

![indiaagriculture](./ef.PNG)

## State efficiency by Season

<div dir="rtl">
שאלה: האם יש קשר בין המיקום הגיאוגרפי לעונה היעילה ביותר בכל מדינה ?
</div>

In [ ]:
state_most_efficient_season = Crop_efficiency_data[Crop_efficiency_data != "Whole Year"].groupby(["State"]).apply(lambda x: x.groupby("Season").Yield.mean().idxmax())
state_most_efficient_season = state_most_efficient_season.to_frame().reset_index().rename({0:"Season"}, axis= 1)

In [ ]:
px.choropleth_mapbox(state_most_efficient_season, geojson= states,featureidkey= "properties.NAME_1", locations="State",
                     color= "Season",
                        color_continuous_scale="Viridis",
                           range_color=(0, 12),
                           mapbox_style="carto-positron",
                           zoom=3, center = {"lat": 22.736424, "lon":78.473622},
                           opacity=0.5)

<div dir="rtl">
מסקנה: ישנו קשר בין המיקום הגיאוגרפי לעונה היעילה ביותר ליבול בכל מדינה ניתן לראות בגרף שהחלק המערבי יעיל בעונה רבי החלק המרכזי יעיל בקיץ והמזרחי יעיל בחורף 
</div>

![indiaagriculture](./se.PNG)

In [ ]:
px.imshow(Crop_efficiency_data[Crop_efficiency_data.Season != "Whole Year"]\
          .pivot_table(index= "Year", columns= "Season", values= "Yield").corr(), text_auto=True)

https://www.kaggle.com/datasets/pyatakov/india-agriculture-crop-production?resource=download
About Dataset
India District-wise agriculture crop area and production statistics 1997-2021
Latest update 02.2023.
Next update of dataset is scheduled in Summer 2023.

Data collected from [Ministry of Agriculture and Farmers Welfare of India](https://aps.dac.gov.in/)

נעזרנו באח של קוליה אהרון - פורת אהרון על מנת להפיק את הגרפים של המפות